In [70]:
import os
#os.getcwd()
os.chdir("/Users/ying/2018Fall/CS539/Project/CS539_ML-master-2")

In [71]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_selection import f_classif
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

## read file

In [72]:
processed_train_df = pd.read_csv("processed_train_df.csv", dtype={'fullVisitorId': 'str'})
processed_test_df = pd.read_csv("processed_test_df.csv", dtype={'fullVisitorId': 'str'})

In [73]:
processed_train_df = processed_train_df.drop('Unnamed: 0', axis=1)
processed_test_df = processed_test_df.drop('Unnamed: 0', axis=1)

In [74]:
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


### 5-fold

In [5]:
import random

In [6]:
unique_visitorId = processed_train_df['fullVisitorId'].unique()
random.seed(123)
random.shuffle(unique_visitorId)
no_cust = len(unique_visitorId)
print(no_cust)

715119


In [7]:
fold = 5
id_cv = []
for i in range(fold):
    if i<fold-1:
        cur_cv = unique_visitorId[i*(no_cust//5):(i+1)*(no_cust//5)]
    else:
        cur_cv = unique_visitorId[i*(no_cust//5):no_cust]
    id_cv.append(cur_cv)   

### Run function for MSE and Testset prediction

In [8]:
def getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred):
    revenue = np.exp(log_y_tr_pred) - 1
    id_list = list(train['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    
    
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_tr_pred = list(submit['PredictedLogRevenue'])
    
    y_train_sumrev = pd.DataFrame(train.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_train_sumrev['totals.transactionRevenue'] = np.log1p(y_train_sumrev['totals.transactionRevenue'])
    y_tr = list(y_train_sumrev['totals.transactionRevenue'])
    
    mse_tr = mean_squared_error(y_tr, y_tr_pred)
    print('train_mse', mse_tr)
    print('train_rmse', np.sqrt(mse_tr))

    
    revenue = np.exp(log_y_val_pred) - 1
    id_list = list(val['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_val_pred = list(submit['PredictedLogRevenue'])
    
    y_val_sumrev = pd.DataFrame(val.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_val_sumrev['totals.transactionRevenue'] = np.log1p(y_val_sumrev['totals.transactionRevenue'])
    y_val = list(y_val_sumrev['totals.transactionRevenue'])
    
    mse_val = mean_squared_error(y_val, y_val_pred)
    
    print('val_mse', mse_val)
    print('val_rmse', np.sqrt(mse_val))
    return mse_tr, mse_val



In [9]:
def testset_pred(model, filename):
    x_tr = processed_train_df.iloc[:,2:]
    y_tr = processed_train_df.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    model.fit(x_tr, log_y_tr)
    
    x_te = processed_test_df.iloc[:,1:]
    te_id = processed_test_df.iloc[:,0]
    y_te_pred = model.predict(x_te)
    y_te_pred =[0 if i < 0 else i for i in y_te_pred]
    revenue = np.exp(y_te_pred) - 1
    
    # Write submit file
    d = {'fullVisitorId':te_id, 'PredictedLogRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedLogRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedLogRevenue"].sum().reset_index())
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedLogRevenue'])
    print(submit.shape)
    submit.head()
    submit.to_csv(filename+'.csv', index=False)


### Linear regression

In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [11]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model = LinearRegression().fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))





fold: 0


/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning:

internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.



train_mse 3.3684043987922947
train_rmse 1.8353213339337324
val_mse 3.3618812552444544
val_rmse 1.8335433606120293


fold: 1
train_mse 3.3695894074697774
train_rmse 1.8356441396604564
val_mse 3.3753928659491725
val_rmse 1.8372242285440208


fold: 2
train_mse 3.3419278283946707
train_rmse 1.8280940425466823
val_mse 3.472337652773089
val_rmse 1.8634209542594204


fold: 3
train_mse 3.389780861931751
train_rmse 1.8411357532598596
val_mse 3.28216815602054
val_rmse 1.8116755106863205


fold: 4
train_mse 3.3693015781098445
train_rmse 1.8355657378884158
val_mse 3.3503185469525985
val_rmse 1.8303875400998004


Average:
train_mse_5fold 3.3678008149396677
train_rmse_5fold 1.8351522014578294
val_mse_5fold 3.3684196953879706
val_rmse_5fold 1.8352503188403184


In [12]:
# Testset prediction
model = LinearRegression()
filename = 'linreg'
testset_pred(model, filename)

(620627, 2)


### Polynomial Regression

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [14]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model_pipeline = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
    model = model_pipeline.fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))






fold: 0
train_mse 2.9497010358124673
train_rmse 1.717469369686827
val_mse 2.9616451774717647
val_rmse 1.720943106982844


fold: 1
train_mse 2.9538613184556395
train_rmse 1.7186801094024564
val_mse 2.9533518599314004
val_rmse 1.7185318908683076


fold: 2
train_mse 2.9269965318648765
train_rmse 1.7108467295070229
val_mse 3.046064821743657
val_rmse 1.7452979177617949


fold: 3
train_mse 2.9721111892744485
train_rmse 1.723981203283391
val_mse 2.8837536059120024
val_rmse 1.6981618314848566


fold: 4
train_mse 2.9511462404905044
train_rmse 1.7178900548319453
val_mse 2.941866368620516
val_rmse 1.7151869777433935


Average:
train_mse_5fold 2.9507632631795873
train_rmse_5fold 1.7177734933423285
val_mse_5fold 2.957336366735868
val_rmse_5fold 1.7196243449682391


In [15]:
# Testset prediction
model = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
filename = 'polyreg'
testset_pred(model, filename)

(620627, 2)


### Regression Tree

In [16]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [17]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model = DecisionTreeRegressor(max_depth=10)
    model.fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))



fold: 0
train_mse 2.5083886378369926
train_rmse 1.5837893287419866
val_mse 2.853865880651824
val_rmse 1.689338888634197


fold: 1
train_mse 2.50202041874724
train_rmse 1.5817776135561028
val_mse 2.918815880509539
val_rmse 1.708454237171584


fold: 2
train_mse 2.4930431031395335
train_rmse 1.5789373335061572
val_mse 2.935511956004627
val_rmse 1.7133335798975713


fold: 3
train_mse 2.5067081939924396
train_rmse 1.5832587261696807
val_mse 2.8228801285623755
val_rmse 1.6801428893288735


fold: 4
train_mse 2.5076751662474392
train_rmse 1.5835640707743528
val_mse 2.8455934228347517
val_rmse 1.6868886812219566


Average:
train_mse_5fold 2.503567103992729
train_rmse_5fold 1.582265414549656
val_mse_5fold 2.8753334537126234
val_rmse_5fold 1.6956316552508366


In [18]:
# Testset prediction
model = DecisionTreeRegressor(max_depth=10)
filename = 'regtree'
testset_pred(model, filename)

(620627, 2)


### Pre-classified Regression

In [75]:
processed_train_df['clf_label'] = np.where(processed_train_df['totals.transactionRevenue']==0.0, 'zero', 'non-zero')
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source,clf_label
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208,zero
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208,zero
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208,zero
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208,zero
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208,zero


In [80]:
# nonzero_sample = processed_train_df.loc[processed_train_df[processed_train_df['totals.transactionRevenue'] != 0.0].index]
# # nonzero_sample.head()
# zero_indices = processed_train_df[processed_train_df['totals.transactionRevenue'] == 0.0].index
# random_indices = np.random.choice(zero_indices, 11515, replace=False)
# zero_sample = processed_train_df.loc[random_indices]
# print("Zero Samples: " + str(zero_sample.shape))
# print("Non-Zero Samples: " + str(nonzero_sample.shape))

# undersampled_train_df = pd.concat([nonzero_sample, zero_sample])
# print("Undersampled Trainning Set: " + str(undersampled_train_df.shape))
# undersampled_train_df.head()

(11515, 23)
Zero Samples: (11515, 23)
Non-Zero Samples: (11515, 23)
Undersampled Trainning Set: (23030, 23)


,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source,clf_label
752,6194193421514403509,37860000.0,11,11.0,1,8,19,1,1,2,6,38,218,30,0,255,12,4,3327,0,3196,0,non-zero
753,5327166854580374902,306670000.0,11,10.0,3,8,19,1,0,4,7,579,218,75,0,280,12,4,11,5,3196,208,non-zero
799,8885051388942907862,68030000.0,13,11.0,7,8,14,1,0,6,6,579,218,75,0,280,12,4,3327,6,0,358,non-zero
802,185467632009737931,26250000.0,13,12.0,6,8,19,1,0,6,20,540,218,100,0,73,12,4,3327,6,0,358,non-zero
859,3244885836845029978,574150000.0,17,14.0,4,8,14,1,0,6,7,955,218,122,0,482,12,4,3327,6,0,358,non-zero


In [77]:
unique_visitorId = processed_train_df['fullVisitorId'].unique()
random.seed(123)
random.shuffle(unique_visitorId)
no_cust = len(unique_visitorId)
print(no_cust)

715119


In [78]:
fold = 5
id_cv = []
for i in range(fold):
    if i<fold-1:
        cur_cv = unique_visitorId[i*(no_cust//5):(i+1)*(no_cust//5)]
    else:
        cur_cv = unique_visitorId[i*(no_cust//5):no_cust]
    id_cv.append(cur_cv) 

### Classifiers

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

feature_list = [k for k in list(undersampled_train_df) if k not in ['fullVisitorId', 'totals.transactionRevenue', 'clf_label']]
LR_accuracy = []
KNN_accuracy = []
DT_accuracy = []
SVM_accuracy = []
RF_accuracy = []

for i in range(fold):
    val = undersampled_train_df[undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    train = undersampled_train_df[~undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train[feature_list]
    y_clf_tr = train['clf_label']
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    
    # --- Logistics Regression -----
    logreg = LogisticRegression()
    logreg.fit(x_tr, y_clf_tr)
    y_clf_pred = logreg.predict(x_val)
    score = logreg.score(x_val, y_clf_val)
    LR_accuracy.append(score)
print('Logistics Regression')
print('Average accuracy: {:.5f}\n\n'.format(sum(LR_accuracy)/len(LR_accuracy)))

for i in range(fold):
    val = undersampled_train_df[undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    train = undersampled_train_df[~undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train[feature_list]
    y_clf_tr = train['clf_label']
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    
    # --- KNN -----
    KNN = KNeighborsClassifier(n_neighbors=15)
    KNN.fit(x_tr, y_clf_tr)
    y_clf_pred = KNN.predict(x_val)
    score = KNN.score(x_val, y_clf_val)
    KNN_accuracy.append(score)
print("KNN")
print('Average accuracy: {:.5f}\n\n'.format(sum(KNN_accuracy)/len(KNN_accuracy)))

for i in range(fold):
    val = undersampled_train_df[undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    train = undersampled_train_df[~undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train[feature_list]
    y_clf_tr = train['clf_label']
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    
    # --- Decision Tree -----
    DT = DecisionTreeClassifier()
    DT.fit(x_tr, y_clf_tr)
    y_clf_pred = DT.predict(x_val)
    score = DT.score(x_val, y_clf_val)
    DT_accuracy.append(score)
print("Decision Tree")
print('Average accuracy: {:.5f}\n\n'.format(sum(DT_accuracy)/len(DT_accuracy)))

for i in range(fold):
    val = undersampled_train_df[undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    train = undersampled_train_df[~undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train[feature_list]
    y_clf_tr = train['clf_label']
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    
    # --- SVM -----
    SVM = SVC()
    SVM.fit(x_tr, y_clf_tr)
    y_clf_pred = SVM.predict(x_val)
    score = SVM.score(x_val, y_clf_val)
    SVM_accuracy.append(score)
print("SVM")
print('Average accuracy: {:.5f}\n\n'.format(sum(SVM_accuracy)/len(SVM_accuracy)))

for i in range(fold):
    val = undersampled_train_df[undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    train = undersampled_train_df[~undersampled_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train[feature_list]
    y_clf_tr = train['clf_label']
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    
    # --- Random Forest -----
    RF = RandomForestClassifier(n_estimators=100)
    RF.fit(x_tr, y_clf_tr)
    y_clf_pred = RF.predict(x_val)
    score = RF.score(x_val, y_clf_val)
    RF_accuracy.append(score)
print("Random Forest")
print('Average accuracy: {:.5f}\n\n'.format(sum(SVM_accuracy)/len(SVM_accuracy)))

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



Logistics Regression
Average accuracy: 0.94709


KNN
Average accuracy: 0.86148


Decision Tree
Average accuracy: 0.92990


SVM
Average accuracy: 0.75035


Random Forest
Average accuracy: 0.75035




## Logistics Regression + Regression Tree

In [83]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []
feature_list = [k for k in list(processed_train_df) if k not in ['fullVisitorId', 'totals.transactionRevenue', 'clf_label']]

for i in range(fold):
    print('\n\nfold:', i)
 
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    nonzero_sample = train.loc[train[train['totals.transactionRevenue'] != 0.0].index]
    zero_indices = train[train['totals.transactionRevenue'] == 0.0].index
    random_indices = np.random.choice(zero_indices, nonzero_sample.shape[0], replace=False)
    zero_sample = train.loc[random_indices]
    print("Zero Samples: " + str(zero_sample.shape))
    print("Non-Zero Samples: " + str(nonzero_sample.shape))

    undersampled_train_df = pd.concat([nonzero_sample, zero_sample])
    print("Undersampled Trainning Set: " + str(undersampled_train_df.shape))
#     undersampled_train_df.head()

    x_tr = undersampled_train_df[feature_list]
    y_clf_tr = undersampled_train_df['clf_label']
    y_tr = undersampled_train_df.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    
    # ----- Classification -----
    nonzero_index_tr = []
    nonzero_index_val = []
    logreg = LogisticRegression()
    logreg.fit(x_tr, y_clf_tr)   
    y_clf_tr_pred = logreg.predict(x_tr)
    y_clf_val_pred = logreg.predict(x_val)
    
    for m in range(len(y_clf_tr_pred)):
        if y_clf_tr_pred[m] == 'zero':
            continue
        else:
            nonzero_index_tr.append(m)
    
    x_regr_tr = x_tr.iloc[nonzero_index_tr]
    y_regr_tr = train.iloc[nonzero_index_tr,1]
    log_y_tr = np.log1p(y_regr_tr)
    
    for j in range(len(y_clf_val_pred)):
        if y_clf_val_pred[j] == 'zero':
            continue
        else:
            nonzero_index_val.append(j)
    
    x_regr_val = x_val.iloc[nonzero_index_val]
    y_regr_val = train.iloc[nonzero_index_val,1]
    log_y_val = np.log1p(y_regr_val)
    
    
    # ----- Regression -----
    model = DecisionTreeRegressor(max_depth=10)
    model.fit(x_regr_tr, log_y_tr)
    
    log_y_tr_pred = model.predict(x_regr_tr)
    tr_pred = list(0 for i in range(len(x_tr)))
    num = 0
    for index in nonzero_index_tr:
        tr_pred[index] = log_y_tr_pred[num]
        num += 1
    tr_pred = [0 if i < 0 else i for i in tr_pred]
    
    log_y_val_pred = model.predict(x_regr_val)
    val_pred = list(0 for i in range(len(x_val)))
    num = 0
    for index in nonzero_index_val:
        val_pred[index] = log_y_val_pred[num]
        num += 1
    val_pred = [0 if i < 0 else i for i in val_pred]
    
    mse_tr, mse_val = getMse(x_tr, undersampled_train_df, val, tr_pred, val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))



fold: 0
Zero Samples: (9236, 23)
Non-Zero Samples: (9236, 23)
Undersampled Trainning Set: (18472, 23)
train_mse 147.41064062641664
train_rmse 12.141278376942712
val_mse 4.510249977901685
val_rmse 2.1237349123423304


fold: 1
Zero Samples: (9162, 23)
Non-Zero Samples: (9162, 23)
Undersampled Trainning Set: (18324, 23)
train_mse 147.22828825913163
train_rmse 12.133766449834596
val_mse 4.470408342330613
val_rmse 2.114334018628706


fold: 2
Zero Samples: (9179, 23)
Non-Zero Samples: (9179, 23)
Undersampled Trainning Set: (18358, 23)
train_mse 147.1601248242403
train_rmse 12.130957292161254
val_mse 4.574714531241092
val_rmse 2.1388582307486144


fold: 3
Zero Samples: (9230, 23)
Non-Zero Samples: (9230, 23)
Undersampled Trainning Set: (18460, 23)
train_mse 147.55885183677154
train_rmse 12.147380451635305
val_mse 4.3435153958630215
val_rmse 2.0841102168222827


fold: 4
Zero Samples: (9253, 23)
Non-Zero Samples: (9253, 23)
Undersampled Trainning Set: (18506, 23)
train_mse 147.53439168766948


## Decision Tree + Regression Tree

In [84]:
from sklearn.tree import DecisionTreeClassifier

train_mse = []
train_rmse = []
val_mse = []
val_rmse = []
feature_list = [k for k in list(processed_train_df) if k not in ['fullVisitorId', 'totals.transactionRevenue', 'clf_label']]

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    
    x_val = val[feature_list]
    y_clf_val = val['clf_label']
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    nonzero_sample = train.loc[train[train['totals.transactionRevenue'] != 0.0].index]
    zero_indices = train[train['totals.transactionRevenue'] == 0.0].index
    random_indices = np.random.choice(zero_indices, nonzero_sample.shape[0], replace=False)
    zero_sample = train.loc[random_indices]
    print("Zero Samples: " + str(zero_sample.shape))
    print("Non-Zero Samples: " + str(nonzero_sample.shape))

    undersampled_train_df = pd.concat([nonzero_sample, zero_sample])
    print("Undersampled Trainning Set: " + str(undersampled_train_df.shape))

    x_tr = undersampled_train_df[feature_list]
    y_clf_tr = undersampled_train_df['clf_label']
    y_tr = undersampled_train_df.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    
    # ----- Classification -----
    nonzero_index_tr = []
    nonzero_index_val = []
    model = DecisionTreeClassifier()
    model.fit(x_tr, y_clf_tr)   
    y_clf_tr_pred = model.predict(x_tr)
    y_clf_val_pred = model.predict(x_val)
    
    for m in range(len(y_clf_tr_pred)):
        if y_clf_tr_pred[m] == 'zero':
            continue
        else:
            nonzero_index_tr.append(m)
    
    x_regr_tr = x_tr.iloc[nonzero_index_tr]
    y_regr_tr = train.iloc[nonzero_index_tr,1]
    log_y_tr = np.log1p(y_regr_tr)
    
    for j in range(len(y_clf_val_pred)):
        if y_clf_val_pred[j] == 'zero':
            continue
        else:
            nonzero_index_val.append(j)
    
    x_regr_val = x_val.iloc[nonzero_index_val]
    y_regr_val = train.iloc[nonzero_index_val,1]
    log_y_val = np.log1p(y_regr_val)
    
    
    # ----- Regression -----
    model = DecisionTreeRegressor(max_depth=10)
    model.fit(x_regr_tr, log_y_tr)
    
    log_y_tr_pred = model.predict(x_regr_tr)
    tr_pred = list(0 for i in range(len(x_tr)))
    num = 0
    for index in nonzero_index_tr:
        tr_pred[index] = log_y_tr_pred[num]
        num += 1
    tr_pred = [0 if i < 0 else i for i in tr_pred]
    
    log_y_val_pred = model.predict(x_regr_val)
    val_pred = list(0 for i in range(len(x_val)))
    num = 0
    for index in nonzero_index_val:
        val_pred[index] = log_y_val_pred[num]
        num += 1
    val_pred = [0 if i < 0 else i for i in val_pred]
    
    mse_tr, mse_val = getMse(x_tr, undersampled_train_df, val, tr_pred, val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))



fold: 0
Zero Samples: (9236, 23)
Non-Zero Samples: (9236, 23)
Undersampled Trainning Set: (18472, 23)
train_mse 147.3084019172115
train_rmse 12.137067270029094
val_mse 4.6512306562807995
val_rmse 2.1566711979995468


fold: 1
Zero Samples: (9162, 23)
Non-Zero Samples: (9162, 23)
Undersampled Trainning Set: (18324, 23)
train_mse 147.34000686663308
train_rmse 12.138369201282067
val_mse 4.66631597348692
val_rmse 2.1601657282456177


fold: 2
Zero Samples: (9179, 23)
Non-Zero Samples: (9179, 23)
Undersampled Trainning Set: (18358, 23)
train_mse 147.02256744531218
train_rmse 12.125286283024916
val_mse 4.632853753702423
val_rmse 2.152406502894475


fold: 3
Zero Samples: (9230, 23)
Non-Zero Samples: (9230, 23)
Undersampled Trainning Set: (18460, 23)
train_mse 147.54692790520195
train_rmse 12.146889639129926
val_mse 4.42092288592275
val_rmse 2.102599078741059


fold: 4
Zero Samples: (9253, 23)
Non-Zero Samples: (9253, 23)
Undersampled Trainning Set: (18506, 23)
train_mse 147.00612642916246
tra